In [ ]:
import os
import io
import numpy as np
import matplotlib.pyplot as plt
import duckdb
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrowfs_adlgen2 as pa_adl

from dotenv import load_dotenv
from pathlib import Path

from azure.identity import DefaultAzureCredential, ClientSecretCredential
from azure.storage.filedatalake import DataLakeServiceClient
from azure.keyvault.secrets import SecretClient
from azure.core.exceptions import ResourceNotFoundError

# local

In [ ]:
# DATA_DIR = "../include/data"

In [ ]:
# # local
# BRONZE_FOLDER_NAME = "bronze"
# BRONZE_DATA_DIR = os.path.join("{DATA_DIR}", "{FOLDER_NAME}").replace("\\", "/")
# BRONZE_DATA_DIR

In [ ]:
# # local
# file_infos = [
#     os.path.join(BRONZE_DATA_DIR.format(DATA_DIR=DATA_DIR, FOLDER_NAME=BRONZE_FOLDER_NAME), file_info).replace("\\", "/") 
#     for file_info in 
#     os.listdir(BRONZE_DATA_DIR.format(DATA_DIR=DATA_DIR, FOLDER_NAME=BRONZE_FOLDER_NAME))
# ]
# file_infos

In [ ]:
# label_paths = [
#     os.path.join(file_info, "etc", "README").replace("\\", "/") 
#     for file_info in file_infos 
#     if os.path.exists(os.path.join(file_info, "etc", "README").replace("\\", "/"))
# ]
# label_paths

# cloud (which will includde connecting to azure data lake storage via duck db)

In [ ]:
# # Retrieve credentials from environment variables
# # this is strictly used only in development
# # load env variables
# env_dir = Path('../../').resolve()
# load_dotenv(os.path.join(env_dir, '.env'))

In [ ]:
storage_account_name = os.environ.get("STORAGE_ACCOUNT_NAME")
credential = os.environ.get("STORAGE_ACCOUNT_KEY")
conn_str = os.environ.get("STORAGE_ACCOUNT_CONN_STR")

In [ ]:
# cloud
URL = "abfss://{STORAGE_ACCOUNT_NAME}.dfs.core.windows.net/{FOLDER_NAME}"
BRONZE_FOLDER_NAME = f"{storage_account_name}-bronze"
BRONZE_DATA_DIR = URL.format(
    STORAGE_ACCOUNT_NAME=storage_account_name,
    FOLDER_NAME=BRONZE_FOLDER_NAME
)
BRONZE_DATA_DIR

In [ ]:
# cloud
# create client with generated sas token
datalake_service_client = DataLakeServiceClient(
    account_url=f"https://{storage_account_name}.dfs.core.windows.net", 
    credential=credential
)

# retrieves file system client/container client 
# to retrieve datalake client
bronze_container_client = datalake_service_client.get_file_system_client(f"{storage_account_name}-bronze")

# we only get the directories in the first level of 
# the container, if it has a "/" then it means it is not
# an immediate folder in the container. This only really
# gets the subject folders 
subject_folders = [path.name for path in bronze_container_client.get_paths() if not "/" in path.name]

In [ ]:
file_infos = [os.path.join(BRONZE_DATA_DIR, subject_folder).replace("\\", "/") for subject_folder in subject_folders]
file_infos

In [ ]:
# need some way to ensure the path also exists
label_paths = [
    os.path.join(file_info, "etc", "README").replace("\\", "/") 
    for i, file_info in enumerate(file_infos) 
    if bronze_container_client
    .get_file_client(file_path=f"{subject_folders[i]}/etc/README")
    .exists()
]
label_paths

In [ ]:
conn = duckdb.connect()

In [ ]:
# for cloud only
# installing dependencies and creating secrets object
conn.sql(f"""INSTALL azure""")
conn.sql(f"""LOAD azure""")
conn.sql(f"""
    CREATE OR REPLACE SECRET az_sgp (
        TYPE azure,
        CONNECTION_STRING '{conn_str}'
    );
""")
# the is required if this notebook is run in linux environment
# like airflow container
conn.sql("SET azure_transport_option_type = 'curl'")

In [ ]:
conn.sql(f"""
    CREATE OR REPLACE TEMPORARY TABLE split_raw_labels AS (
        WITH raw_labels AS (
            SELECT 
                filename, 
                content 
            FROM read_text({label_paths})
        )

        SELECT 
            filename, 
            REGEXP_SPLIT_TO_TABLE(content, '\n\n*') AS value 
        FROM raw_labels
    )
""")

In [ ]:
conn.sql("""
    SELECT *  FROM split_raw_labels 
""")

# clean value column containing the label

In [ ]:
conn.sql("""
    CREATE OR REPLACE TEMPORARY TABLE subjects_labels AS (
        -- remove the rows with no gender meta data of the subject
        WITH subjects_labels_1 AS (
            SELECT * 
            FROM split_raw_labels
            WHERE 'gender' IN LCASE(value)
        ),

        subjects_labels_2 AS (
            SELECT
                -- remove punctuations in the meta data of the subject
                -- containing its gender
                REGEXP_REPLACE(LCASE(value), '[:;\[\]\t\n\s]+', '') AS value,
                filename
            FROM subjects_labels_1
        ),

        subjects_labels_3 AS (
            SELECT
                -- remove 'gender' in meta data of the subject containing
                -- its gender
                REGEXP_REPLACE(value, 'gender', '') AS value,
                filename
            FROM subjects_labels_2
        ),
            
        subjects_labels_4 AS (
            SELECT
                CASE
                    WHEN STARTS_WITH(value, 'ma') OR STARTS_WITH(value, 'mä') THEN 'male'
                    WHEN STARTS_WITH(value, 'fem') OR STARTS_WITH(value, 'wei') THEN 'female'
            
                    -- whenever unknown gender is encountered just generate random
                    -- gender of either male or female with prob of 50%
                    ELSE 
                        (CASE 
                            WHEN RANDOM() < 0.5 THEN 'male'
                            ELSE 'female'
                        END)
                END AS value,
                filename
            FROM subjects_labels_3
        )
            
        SELECT * FROM subjects_labels_4
    )
""")

In [ ]:
conn.sql("""
    SELECT * FROM subjects_labels
""")

# clean filename column and turn into subjectid column

In [ ]:
conn.sql("""
    CREATE OR REPLACE TEMPORARY TABLE subjects_labels AS (
        SELECT
            -- splits the file path on the '/' character which
            -- results in a list that we can use to extract the
            -- third to the last value in this list
            LIST_EXTRACT(STRING_SPLIT(filename, '/'), -3) AS subjectId,
            value
        FROM subjects_labels
    )
""")

In [ ]:
conn.sql("""
    SELECT * FROM subjects_labels
""")

# once the `value`'s and `subjectId`'s is cleaned we can now split the dataset uniformly. We need to split first on the male and female subjects so that we can evenly split these male and female sets into train, val, and test sets, and then later rejoin the train, val, and test sets of the male and female sets  

In [ ]:
conn.sql("""
    CREATE OR REPLACE TEMPORARY TABLE male_subjects_labels AS (
        SELECT 
            *, 
            ROW_NUMBER() OVER(ORDER BY subjectId) AS split_row_id 
        FROM subjects_labels
        WHERE value = 'male'
    );
    
    CREATE OR REPLACE TEMPORARY TABLE female_subjects_labels AS (
        SELECT 
            *, 
            ROW_NUMBER() OVER(ORDER BY subjectId) AS split_row_id 
        FROM subjects_labels
        WHERE value = 'female'
    );    
""")

# create a 70, 15, 15 ratio for the training, validation, and testing sets

In [ ]:
# creates 70, 15, 15 ratio for male labels
conn.sql("""
    CREATE OR REPLACE TEMPORARY TABLE male_subjects_labels_split AS (
        -- split the male subjects labels into train and test 
        WITH male_subjects_labels_train__ AS (
            SELECT 
                *,
                CASE
                    WHEN split_row_id <= (SELECT COUNT(*) * 0.7 FROM male_subjects_labels) THEN 'train'
                    ELSE 'test'
                END AS split
            FROM male_subjects_labels
        ),
        
        -- recalculate the test splits split row id so it 
        -- starts from 1 to n again 
        temp AS (
            SELECT 
                * EXCLUDE(split_row_id), 
                ROW_NUMBER() OVER(ORDER BY subjectId) AS split_row_id
            FROM male_subjects_labels_train__
            WHERE split = 'test'
        ),
        
        -- split again the previous test set into now the real
        -- validation and testing sets
        male_subjects_labels_val_test AS (
            SELECT
                * EXCLUDE(split),
                CASE
                    WHEN split_row_id <= (SELECT COUNT(*) * 0.5 FROM temp) THEN 'val'
                    ELSE 'test'
                END AS split
            FROM temp
        )
        
        -- unionize the male subjects labels with validation
        -- and testing splits and the previous male subjects
        -- labels with the train splits 
        SELECT * FROM male_subjects_labels_val_test
        UNION BY NAME
        SELECT * FROM male_subjects_labels_train__
        WHERE split = 'train'
    )
""")

In [ ]:
conn.sql("""
    SELECT * FROM male_subjects_labels_split
""")

In [ ]:
# creates 70, 15, 15 ratio for female labels
conn.sql("""
    CREATE OR REPLACE TEMPORARY TABLE female_subjects_labels_split AS (
        -- split the female subjects labels into train and test 
        WITH female_subjects_labels_train__ AS (
            SELECT 
                *,
                CASE
                    WHEN split_row_id <= (SELECT COUNT(*) * 0.7 FROM female_subjects_labels) THEN 'train'
                    ELSE 'test'
                END AS split
            FROM female_subjects_labels
        ),
        
        -- recalculate the test splits split row id so it 
        -- starts from 1 to n again 
        temp AS (
            SELECT 
                * EXCLUDE(split_row_id), 
                ROW_NUMBER() OVER(ORDER BY subjectId) AS split_row_id
            FROM female_subjects_labels_train__
            WHERE split = 'test'
        ),
        
        -- split again the previous test set into now the real
        -- validation and testing sets
        female_subjects_labels_val_test AS (
            SELECT
                * EXCLUDE(split),
                CASE
                    WHEN split_row_id <= (SELECT COUNT(*) * 0.5 FROM temp) THEN 'val'
                    ELSE 'test'
                END AS split
            FROM temp
        )
        
        -- unionize the female subjects labels with validation
        -- and testing splits and the previous female subjects
        -- labels with the train splits 
        SELECT * FROM female_subjects_labels_val_test
        UNION BY NAME
        SELECT * FROM female_subjects_labels_train__
        WHERE split = 'train'
    )
""")

In [ ]:
conn.sql("""
    SELECT * FROM female_subjects_labels_split
""")

# rejoin/unionize the male and female subjects labels based on their split as well as recreate new `rowId`'s for each split and create a partition column for each split

In [ ]:
n_partitions = 10

In [ ]:
conn.sql(f"""
    CREATE OR REPLACE TEMPORARY TABLE train_labels AS (
        WITH subjects_labels_split AS (
            SELECT subjectId, value, split
            FROM female_subjects_labels_split
            WHERE split = 'train'
            
            UNION BY NAME
            
            SELECT subjectId, value, split
            FROM male_subjects_labels_split
            WHERE split = 'train'
        ),
         
        -- create new rowIds for the split
        train_labels AS (
            SELECT
                *,
                ROW_NUMBER() OVER(ORDER BY subjectId) - 1 AS rowId
            FROM subjects_labels_split
        )
         
        -- creawte partition column
        SELECT
            *,
            rowId % {n_partitions} AS partition
        FROM train_labels
    );
""")

In [ ]:
train_labels_table = conn.sql("""
    SELECT * FROM train_labels
""").to_arrow_table()
train_labels_table

In [ ]:
conn.sql(f"""
    CREATE OR REPLACE TEMPORARY TABLE val_labels AS (
        WITH subjects_labels_split AS (
            SELECT subjectId, value, split
            FROM female_subjects_labels_split
            WHERE split = 'val'
            
            UNION BY NAME
            
            SELECT subjectId, value, split
            FROM male_subjects_labels_split
            WHERE split = 'val'
        ),
         
        -- create new rowIds for the split
        val_labels AS (
            SELECT
                *,
                ROW_NUMBER() OVER(ORDER BY subjectId) - 1 AS rowId
            FROM subjects_labels_split
        )
         
        -- creawte partition column
        SELECT
            *,
            rowId % {n_partitions} AS partition
        FROM val_labels
    );
""")

In [ ]:
val_labels_table = conn.sql("""         
    SELECT * FROM val_labels
""").to_arrow_table()
val_labels_table

In [ ]:
conn.sql(f"""
    CREATE OR REPLACE TEMPORARY TABLE test_labels AS (
        WITH subjects_labels_split AS (
            SELECT subjectId, value, split
            FROM female_subjects_labels_split
            WHERE split = 'test'
            
            UNION BY NAME
            
            SELECT subjectId, value, split
            FROM male_subjects_labels_split
            WHERE split = 'test'
        ),
         
        -- create new rowIds for the split
        test_labels AS (
            SELECT
                *,
                ROW_NUMBER() OVER(ORDER BY subjectId) - 1 AS rowId
            FROM subjects_labels_split
        )
         
        -- creawte partition column
        SELECT
            *,
            rowId % {n_partitions} AS partition
        FROM test_labels
    );
""")

In [ ]:
test_labels_table = conn.sql("""
    SELECT * FROM test_labels
""").to_arrow_table()
test_labels_table

# save the data to azure data lake or locally

In [ ]:
# # local
# SILVER_FOLDER_NAME = "silver"
# SUB_FOLDER_NAME = "stage-01"
# SILVER_DATA_DIR = os.path.join("{DATA_DIR}", "{FOLDER_NAME}", "{SUB_FOLDER_NAME}").replace("\\", "/")
# SILVER_DATA_DIR

In [ ]:
# SAVE_DIR = SILVER_DATA_DIR.format(
#     DATA_DIR=DATA_DIR,
#     FOLDER_NAME=SILVER_FOLDER_NAME,
#     SUB_FOLDER_NAME=SUB_FOLDER_NAME
# )
# os.makedirs(SAVE_DIR, exist_ok=True)
# SAVE_DIR

In [ ]:
# cloud
# URL = "abfss://{FOLDER_NAME}@sgppipelinesa.dfs.core.windows.net"
URL = "{FOLDER_NAME}"
SILVER_FOLDER_NAME = "sgppipelinesa-silver"
SUB_FOLDER_NAME = "stage-01"
SILVER_DATA_DIR = os.path.join(URL, "{SUB_FOLDER_NAME}").replace("\\", "/")
SILVER_DATA_DIR

In [ ]:
SAVE_DIR = SILVER_DATA_DIR.format(
    FOLDER_NAME=SILVER_FOLDER_NAME,
    SUB_FOLDER_NAME=SUB_FOLDER_NAME
)
SAVE_DIR

# Unfortunately writing to azure using duckdb is not possible yet so workaround is to use pyarrow table

In [ ]:
# conn.sql(f"""
#     COPY (SELECT * FROM train_labels) TO '{SAVE_DIR}/train_labels.parquet' (FORMAT parquet, PARTITION_BY partition, OVERWRITE)
# """)

In [ ]:
handler = pa_adl.AccountHandler.from_account_name(storage_account_name, credential=credential)
fs = pa.fs.PyFileSystem(handler)

In [ ]:
pq.write_table(train_labels_table, f"{SAVE_DIR}/train_labels.parquet", filesystem=fs)
pq.write_table(val_labels_table, f"{SAVE_DIR}/val_labels.parquet", filesystem=fs)
pq.write_table(test_labels_table, f"{SAVE_DIR}/test_labels.parquet", filesystem=fs)